In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import imblearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, precision_recall_curve, auc
import scipy.linalg as la

In [3]:
data_path = "/home/sliu375/dat490/diabetes.csv"
df = pd.read_csv(data_path, header = 0)

In [4]:
print(df.shape)
df.head()

(768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [16]:
# Helper function to compute G-mean
def g_mean(y_true, y_pred):
    from sklearn.metrics import recall_score
    sensitivity = recall_score(y_true, y_pred, pos_label=1)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    return np.sqrt(sensitivity * specificity)

# Helper function to compute AUC-PR
def auc_pr(y_true, y_prob):
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    return auc(recall, precision)

# Function to compute IR ratio
def compute_ir_ratio(y):
    counts = np.bincount(y.astype(int))
    if len(counts) < 2:  # Ensure both classes are present
        return float('inf') if counts[0] > 0 else 0
    majority = max(counts)
    minority = min(counts)
    return majority / minority if minority > 0 else float('inf')

# Fisher Information Matrix (Equation 4)
def fisher_information(X, beta, lambda_reg):
    probs = 1 / (1 + np.exp(-X @ beta))
    W = np.diag(probs * (1 - probs))
    I = X.T @ W @ X + lambda_reg * np.eye(X.shape[1])
    return I

# MSEE computation (Equation 7)
def compute_msee(X_j, X_i, beta, lambda_reg, S_X, S_y):
    # Update Fisher information with X_i
    X_S = np.vstack([S_X, X_i])
    I_S_xi = fisher_information(X_S, beta, lambda_reg)
    I_inv = la.inv(I_S_xi)
    
    # Predicted probabilities
    pi_j = 1 / (1 + np.exp(-X_j @ beta))
    
    # Bias term
    bias_term = -pi_j * (1 - pi_j) * (X_j @ (lambda_reg * I_inv @ beta))
    
    # Variance term
    var_term = (pi_j * (1 - pi_j))**2 * (X_j @ I_inv @ (I_S_xi - lambda_reg * np.eye(X_S.shape[1])) @ I_inv.T @ X_j)
    
    return (bias_term**2 + var_term)

# Here we are... Main Active Downsampling Algorithm
def active_downsampling(X, y, l=2, k=200, lambda_reg=1.0, epsilon=0.1, C_size=10):
    # Standardize the features
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    X_std[X_std == 0] = 1  # Avoid division by zero
    X_scaled = (X - X_mean) / X_std

    # (1) Initialize training, pool, and test sets
    n_samples = X_scaled.shape[0]
    indices = np.arange(n_samples)
    
    # Ensure balanced initial training set (l instances per class)
    pos_indices = indices[y == 1]
    neg_indices = indices[y == 0]
    if len(pos_indices) < l or len(neg_indices) < l:
        raise ValueError(f"Not enough instances to select {l} per class. Pos: {len(pos_indices)}, Neg: {len(neg_indices)}")
    pos_indices = pos_indices[:l]
    neg_indices = neg_indices[:l]
    train_indices = np.concatenate([pos_indices, neg_indices])
    S_indices = train_indices
    SC_indices = np.setdiff1d(indices, S_indices)
    
    # Test set: Use 20% of remaining data
    test_size = int(0.2 * len(SC_indices))
    test_indices = SC_indices[:test_size]
    SC_indices = SC_indices[test_size:]
    
    S_X, S_y = X_scaled[S_indices], y[S_indices]
    SC_X, SC_y = X_scaled[SC_indices], y[SC_indices]
    test_X, test_y = X_scaled[test_indices], y[test_indices]
    
    # (2). Iterative selection
    for n in range(k):
        if len(SC_X) < C_size:
            print(f"Pool set too small at iteration {n}. Stopping early.")
            break
            
        # Calculate cost weights (inversely proportional to class frequencies)
        S_pos = np.sum(S_y == 1)
        S_neg = np.sum(S_y == 0)
        c01 = 1 / S_pos if S_pos > 0 else 1
        c10 = 1 / S_neg if S_neg > 0 else 1
        sample_weight = np.where(S_y == 1, c01, c10)
        
        # Estimate parameters (Equation 2) using penalized logistic regression
        model = LogisticRegression(penalty='l2', C=1/lambda_reg, fit_intercept=True, solver='lbfgs', max_iter=1000)
        model.fit(S_X, S_y, sample_weight=sample_weight)
        
        # Extract beta (coefficients and intercept)
        beta = np.concatenate([model.intercept_, model.coef_[0]])
        S_X_with_intercept = np.hstack([np.ones((S_X.shape[0], 1)), S_X])
        SC_X_with_intercept = np.hstack([np.ones((SC_X.shape[0], 1)), SC_X])
        
        # Pre-select candidates (Equation 8)
        probs = model.predict_proba(SC_X)[:, 1]
        uncertainty = np.abs(probs - (1 - probs))
        C_indices = np.argsort(uncertainty)[:min(C_size, len(uncertainty))]
        C_X = SC_X[C_indices]
        C_X_with_intercept = SC_X_with_intercept[C_indices]
        
        # Compute average MSEE for each candidate (Equation 9)
        avg_msee = []
        for i in range(len(C_indices)):
            msee = 0
            for j in range(len(SC_X)):
                msee += compute_msee(SC_X_with_intercept[j], C_X_with_intercept[i], beta, lambda_reg, S_X_with_intercept, S_y)
            avg_msee.append(msee / len(SC_X))
        
        # Select the most informative instance
        best_idx = np.argmin(avg_msee)
        x_star = C_X[best_idx]
        y_star = SC_y[C_indices[best_idx]]
        
        # Update training and pool sets
        S_X = np.vstack([S_X, x_star])
        S_y = np.append(S_y, y_star)
        SC_X = np.delete(SC_X, C_indices[best_idx], axis=0)
        SC_y = np.delete(SC_y, C_indices[best_idx])
        SC_X_with_intercept = np.delete(SC_X_with_intercept, C_indices[best_idx], axis=0)
    
    # Now let's count the distribution of the downsampled training set (S_y)
    downsampled_distribution = np.bincount(S_y.astype(int))
    print("Downsampled class distribution (0 vs 1):", downsampled_distribution)
    print("Resampled IR ratio:", compute_ir_ratio(S_y))
    
    # Final evaluation on test set
    y_pred = model.predict(test_X)
    y_prob = model.predict_proba(test_X)[:, 1]
    f_measure = f1_score(test_y, y_pred)
    g_mean_score = g_mean(test_y, y_pred)
    auc_score = roc_auc_score(test_y, y_prob)
    auc_pr_score = auc_pr(test_y, y_prob)
    
    return {
        "F-measure": f_measure,
        "G-mean": g_mean_score,
        "AUC": auc_score,
        "AUC-PR": auc_pr_score
    }

In [17]:
np.random.seed(42)

# Extract X and y
X = df.drop('Outcome', axis=1).values
y = df['Outcome'].values

# Check original class distribution & IR
print("Class distribution (0 vs 1):", np.bincount(y))
ir_ratio_original = compute_ir_ratio(y)
print("IR ratio of original dataset:", ir_ratio_original)
print("\n")

# Run the active downsampling algorithm
results = active_downsampling(X, y, l=2, k=50, lambda_reg=1.0, epsilon=0.1, C_size=10)
print("Evaluation results:", results)

Class distribution (0 vs 1): [500 268]
IR ratio of original dataset: 1.8656716417910448


Downsampled class distribution (0 vs 1): [28 26]
Resampled IR ratio: 1.0769230769230769
Evaluation results: {'F-measure': 0.5961538461538461, 'G-mean': np.float64(0.6802749433047525), 'AUC': np.float64(0.7877928949357521), 'AUC-PR': np.float64(0.7191333988166677)}
